In [1]:
# Instalação de dependências para manipulação de Parquet e barra de progresso
!pip install pyarrow fastparquet tqdm

In [2]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# =============================================================================
# 1. CONFIGURAÇÃO
# =============================================================================
BASE_DIR = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity"

datasets_targets = {
    "CICIDS2017": os.path.join(BASE_DIR, "CICIDS2017"),
    "CICDDoS2019_Treino": os.path.join(BASE_DIR, "CICDDoS2019", "Treino"),
    "CICDDoS2019_Teste": os.path.join(BASE_DIR, "CICDDoS2019", "Teste")
}

# Colunas conhecidas por terem tipos mistos nesses datasets
# Vamos forçá-las a ser string (object) para evitar crash
COLUNAS_MISTAS = {
    'SimillarHTTP': str,
    'Flow ID': str,
    'Source IP': str,
    'Destination IP': str,
    'Timestamp': str,
    'Label': str
}

# =============================================================================
# 2. FUNÇÃO V3 (COM TRATAMENTO DE SCHEMA E SAFETY-CLOSE)
# =============================================================================
def converter_csv_para_parquet_v3(diretorio_raiz):
    print(f"\n📂 Varrendo: {diretorio_raiz}")
    arquivos_csv = glob.glob(os.path.join(diretorio_raiz, "**", "*.csv"), recursive=True)
    
    if not arquivos_csv:
        print("   ⚠️ Nenhum CSV encontrado.")
        return

    for path_csv in tqdm(arquivos_csv, desc="Convertendo"):
        path_parquet = path_csv.replace('.csv', '.parquet')
        writer = None
        
        try:
            # 1. Limpeza prévia (Se existir arquivo corrompido ou antigo)
            if os.path.exists(path_parquet):
                try:
                    os.remove(path_parquet)
                except OSError:
                    print(f"   ⚠️ Não foi possível remover {os.path.basename(path_parquet)}. Reinicie o Kernel!")
                    continue

            # 2. Leitura com Tipagem Forçada (Evita erro de Schema)
            chunk_iterator = pd.read_csv(
                path_csv, 
                chunksize=100_000, 
                encoding='cp1252', 
                low_memory=False,
                dtype=COLUNAS_MISTAS # <--- O Segredo da correção está aqui
            )
            
            for chunk in chunk_iterator:
                # Limpeza de nomes de colunas
                chunk.columns = chunk.columns.str.strip()
                
                # Remove coluna inútil de índice se existir
                if 'Unnamed: 0' in chunk.columns:
                    chunk.drop(columns=['Unnamed: 0'], inplace=True)
                
                # Garante que as colunas forçadas sejam string mesmo se vierem vazias
                for col in COLUNAS_MISTAS.keys():
                    if col in chunk.columns:
                        chunk[col] = chunk[col].astype(str)

                # Conversão para PyArrow
                table = pa.Table.from_pandas(chunk)
                
                # Inicializa Writer no primeiro chunk
                if writer is None:
                    writer = pq.ParquetWriter(path_parquet, table.schema, compression='snappy')
                
                writer.write_table(table)
                
        except Exception as e:
            print(f"\n❌ ERRO em {os.path.basename(path_csv)}: {e}")
        
        finally:
            # 3. O "Seguro de Vida": Fecha o arquivo não importa o que aconteça
            if writer:
                writer.close()

# =============================================================================
# 3. EXECUÇÃO
# =============================================================================
print(">>> INICIANDO CONVERSÃO V3 (TIPO FORÇADO + SAFETY CLOSE) <<<")

for nome_dataset, caminho in datasets_targets.items():
    print(f"\nProcessing Dataset: {nome_dataset}")
    converter_csv_para_parquet_v3(caminho)

print("\n\n🏁 PROCESSO FINALIZADO! Verifique se os erros sumiram.")

>>> INICIANDO CONVERSÃO V3 (TIPO FORÇADO + SAFETY CLOSE) <<<

Processing Dataset: CICIDS2017

📂 Varrendo: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\CICIDS2017


Convertendo: 100%|███████████████████████████████████████████████████████████████████████| 8/8 [01:06<00:00,  8.37s/it]



Processing Dataset: CICDDoS2019_Treino

📂 Varrendo: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\CICDDoS2019\Treino


Convertendo: 100%|████████████████████████████████████████████████████████████████████| 11/11 [20:14<00:00, 110.44s/it]



Processing Dataset: CICDDoS2019_Teste

📂 Varrendo: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\CICDDoS2019\Teste


Convertendo: 100%|███████████████████████████████████████████████████████████████████████| 7/7 [07:56<00:00, 68.06s/it]



🏁 PROCESSO FINALIZADO! Verifique se os erros sumiram.
